In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
heart_data = pd.read_csv('/content/heart (2).csv')


In [4]:
heart_data.head()

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1

In [5]:
heart_data.shape


(303, 14)

In [6]:
heart_data.isnull().sum()



age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [7]:
heart_data['target'].value_counts()


target
1    165
0    138
Name: count, dtype: int64

1 --> Defective Heart

0 --> Healthy Heart

Splitting the Features and Target

In [10]:
x = heart_data.drop(columns='target', axis=1)
y = heart_data['target']

In [11]:
x = np.asarray(x)
y = np.asarray(y)

Model Selection

1. comparing the models with the default hyperparameter values using cross validation

In [16]:
#list of models
models = [ LogisticRegression(max_iter = 100), SVC(kernel= 'linear'), KNeighborsClassifier(), RandomForestClassifier(random_state = 0)]

In [17]:
def compare_models_cross_validation():
  for model in models :

    cv_score = cross_val_score(model,x ,y, cv = 5)
    mean_Accuracy = sum(cv_score)/len(cv_score)
    mean_Accuracy = mean_Accuracy*100
    mean_Accuracy = round(mean_Accuracy, 3)

    print('cross validation accuracies for the', model,'=', cv_score)
    print('Accuracy score of the ', model, mean_Accuracy)
    print('----------------------------------------------------------')


In [18]:
compare_models_cross_validation()

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

cross validation accuracies for the LogisticRegression() = [0.81967213 0.86885246 0.85245902 0.85       0.75      ]
Accuracy score of the  LogisticRegression() 82.82
----------------------------------------------------------
cross validation accuracies for the SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy score of the  SVC(kernel='linear') 82.831
----------------------------------------------------------
cross validation accuracies for the KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy score of the  KNeighborsClassifier() 64.388
----------------------------------------------------------
cross validation accuracies for the RandomForestClassifier(random_state=0) = [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy score of the  RandomForestClassifier(random_state=0) 83.809
----------------------------------------------------------


2. Comparing the models with different Hyperparameter values using GridSearchCV

In [19]:
# list of model
models_list = [LogisticRegression(max_iter = 1000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state = 0)]

In [20]:
#creating a dictionary that contain hyper parameter vaalues for the above mentioned models


model_hyperparameters = {


    'log_reg_hyperparameters': {

        'C' : [1,5,10,20]
    },

    'svc_hyperparameters': {

        'kernel' : ['linear','poly','rbf','sigmoid'],
        'C' : [1,5,10,20]
    },


    'KNN_hyperparameters' : {

        'n_neighbors' : [3,5,10]
    },


    'random_forest_hyperparameters' : {

        'n_estimators' : [10, 20, 50, 100]
    }
}


In [21]:
print(model_hyperparameters)

{'log_reg_hyperparameters': {'C': [1, 5, 10, 20]}, 'svc_hyperparameters': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}, 'KNN_hyperparameters': {'n_neighbors': [3, 5, 10]}, 'random_forest_hyperparameters': {'n_estimators': [10, 20, 50, 100]}}


In [22]:
print(model_hyperparameters.keys())

dict_keys(['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters'])


In [23]:
model_hyperparameters['log_reg_hyperparameters']


{'C': [1, 5, 10, 20]}

In [25]:
model_keys = list(model_hyperparameters.keys())

In [26]:
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters']


In [27]:
model_keys[0]


'log_reg_hyperparameters'

In [28]:
model_hyperparameters[model_keys[0]]


{'C': [1, 5, 10, 20]}

Applying GridSearchCV



In [29]:
def ModelSelection(list_of_models, hyperparameters_dictionary):

  result = []

  i = 0

  for model in list_of_models:

    key = model_keys[i]

    params = hyperparameters_dictionary[key]

    i += 1

    print(model)
    print(params)
    print('---------------------------------')


    classifier = GridSearchCV(model, params, cv=5)

    # fitting the data to classifier
    classifier.fit(X,Y)

    result.append({
        'model used' : model,
        'highest score' : classifier.best_score_,
        'best hyperparameters' : classifier.best_params_
    })

  result_dataframe = pd.DataFrame(result, columns = ['model used','highest score','best hyperparameters'])

  return result_dataframe


In [30]:
ModelSelection(models_list, model_hyperparameters)


LogisticRegression(max_iter=1000)
{'C': [1, 5, 10, 20]}
---------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
---------------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
---------------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
---------------------------------


model used  highest score  \
0       LogisticRegression(max_iter=1000)       0.831585   
1                                   SVC()       0.828306   
2                  KNeighborsClassifier()       0.643880   
3  RandomForestClassifier(random_state=0)       0.838087   

           best hyperparameters  
0                      {'C': 5}  
1  {'C': 1, 'kernel': 'linear'}  
2            {'n_neighbors': 5}  
3         {'n_estimators': 100}